In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)


In [2]:
deliveries = pd.read_csv(r'F:\t20 data\t20 csv\Cricket_Data\t201.csv')

In [3]:
df = deliveries.copy() # make a copy of the dataframe
df = df.drop(columns=['replacements', 'review_by', 'review_umpire', 'review_batter','review_decision']) # drop unnecessary columns

In [5]:
consistent_teams = ['India', 'Australia', 'England', 'South Africa', 'Pakistan', 'New Zealand', 'West Indies', 'Sri Lanka', 'Bangladesh'] # list of consistent teams

df = df[df['bowling_team'].isin(consistent_teams)] # filter the dataframe for consistent teams
df = df[df['batting_team'].isin(consistent_teams)] # filter the dataframe for consistent teams

In [6]:
df['isdot'] = df['batter_runs'].apply(lambda x: 1 if x == 0 else 0)  # create a new column for dot balls
df['isone'] = df['batter_runs'].apply(lambda x: 1 if x == 1 else 0)  # create a new column for one runs
df['istwo'] = df['batter_runs'].apply(lambda x: 1 if x == 2 else 0)  # create a new column for two runs
df['isthree'] = df['batter_runs'].apply(lambda x: 1 if x == 3 else 0) # create a new column for three runs
df['isfour'] = df['batter_runs'].apply(lambda x: 1 if x == 4 else 0)  # create a new column for four runs
df['isSix'] = df['batter_runs'].apply(lambda x: 1 if x == 6 else 0)  # create a new column for six runs


In [7]:
dots = df.groupby('batter')['isdot'].sum().reset_index().rename(columns={'isdot':'dots'}) # calculate total dot balls faced by each player
  
runs = pd.DataFrame(df.groupby('batter')['batter_runs'].sum()).reset_index().rename(columns={'batter_runs':'runs'}) # calculate total runs scored by each player
  
balls = pd.DataFrame(df.groupby('batter')['ball_number'].count()).reset_index().rename(columns={'ball_number':'balls'}) # calculate total balls faced by each player
  
innings = pd.DataFrame(df.groupby('batter')['match_id'].nunique()).reset_index().rename(columns={'match_id':'innings'}) # calculate total innings played by each player
  
player_dismissed = df.groupby("batter")["wicket_flag"].sum().reset_index().rename(columns={"wicket_flag": "player_dismissed"})
  
sixes = df.groupby('batter')['isSix'].sum().reset_index().rename(columns={'isSix':'sixes'}) # calculate total sixes hit by each player
  
fours = df.groupby('batter')['isfour'].sum().reset_index().rename(columns={'isfour':'fours'}) # calculate total fours hit by each player
  
batterdf = pd.merge(runs, balls, on='batter', how='inner').merge(innings, on='batter', how='inner').merge(player_dismissed, on='batter', how='inner').merge(sixes, on='batter', how='inner').merge(fours, on='batter', how='inner').merge(dots, on='batter', how='inner') # merge all the dataframes

batterdf['RPI'] = batterdf.apply(lambda x: (x['runs']/x['innings']), axis = 1) # calculate runs per innings
batterdf['SR'] = batterdf.apply(lambda x: (x['runs']/x['balls'])*100, axis = 1) # calculate strike rate
 
batterdf["BPB"] = batterdf["balls"] / (batterdf["fours"] + batterdf["sixes"]) # calculate balls per boundary
batterdf["BPD"] = batterdf["balls"] / batterdf["player_dismissed"] # calculate balls per dismissal
batterdf['dot%'] = batterdf["dots"] / batterdf["balls"]# calculate dot ball percentage


In [8]:
def get_phase(over): # function to get the phase of the match
    if over <= 6:
        return 'Powerplay' 
    elif over < 15:
        return 'Middle Overs'
    else:
        return 'Death Overs'

In [9]:
df['phase'] = df['over'].apply(get_phase) # apply the function to get the phase of the match

In [10]:
df.columns

Index(['batter', 'bowler', 'non_striker', 'ball_number', 'batter_runs',
       'extras_runs', 'total_runs', 'wicket_kind', 'player_out', 'wicket_flag',
       'batting_team', 'bowling_team', 'innings', 'over', 'team', 'match_id',
       'venue', 'wides', 'legbyes', 'noballs', 'byes', 'penalty', 'isdot',
       'isone', 'istwo', 'isthree', 'isfour', 'isSix', 'phase'],
      dtype='object')

In [11]:
def balls_per_dismissal(balls, player_dismissed):
    if player_dismissed > 0:
        return balls / player_dismissed  #  Divide by times dismissed
    else:
        return balls / 1  #  Avoid division by zero

def balls_per_boundary(balls, boundaries):
    if boundaries > 0:
        return balls/boundaries
    else:
        return balls/1 

In [ ]:
def ByCustom(df, current_phase, current_innings): 

    # df = df[df['venue'] == current_venue]
    df = df[df['phase'] == current_phase]
    df = df[df['innings'] == current_innings]


    df.reset_index(drop = True, inplace = True)

    df['isdot'] = df['batter_runs'].apply(lambda x: 1 if x == 0 else 0) # create a new column for dot balls
    df['isone'] = df['batter_runs'].apply(lambda x: 1 if x == 1 else 0) # create a new column for one runs
    df['istwo'] = df['batter_runs'].apply(lambda x: 1 if x == 2 else 0) # create a new column for two runs
    df['isthree'] = df['batter_runs'].apply(lambda x: 1 if x == 3 else 0) # create a new column for three runs
    df['isfour'] = df['batter_runs'].apply(lambda x: 1 if x == 4 else 0) # create a new column for four runs
    df['isSix'] = df['batter_runs'].apply(lambda x: 1 if x == 6 else 0) # create a new column for six runs

    runs = pd.DataFrame(df.groupby('batter')['batter_runs'].sum()).reset_index().rename(columns={'batter_runs':'runs'}) # calculate total runs scored by each player

    balls = pd.DataFrame(df.groupby('batter')['ball_number'].count()).reset_index().rename(columns={'ball_number':'balls'}) # calculate total balls faced by each player

    innings = pd.DataFrame(df.groupby('batter')['match_id'].nunique()).reset_index().rename(columns={'match_id':'innings'}) # calculate total innings played by each player

    player_dismissed = df.groupby("batter")["wicket_flag"].sum().reset_index().rename(columns={"wicket_flag": "player_dismissed"}) # calculate total dismissals of each player

    sixes = df.groupby('batter')['isSix'].sum().reset_index().rename(columns={'isSix':'sixes'})  # calculate total sixes hit by each player

    fours = df.groupby('batter')['isfour'].sum().reset_index().rename(columns={'isfour':'fours'}) # calculate total fours hit by each player

    dots = df.groupby('batter')['isdot'].sum().reset_index().rename(columns={'isdot':'dots'})  # calculate total dot balls faced by each player

    batter_df = pd.merge(runs, balls, on='batter', how='inner').merge(innings, on='batter', how='inner').merge(player_dismissed, on='batter', how='inner').merge(sixes, on='batter', how='inner').merge(fours, on='batter', how='inner').merge(dots, on='batter', how='inner')
    
    batter_df['RPI'] = round(batter_df.apply(lambda x: (x['runs']/x['innings']), axis = 1),2) # calculate runs per innings

    batter_df['SR'] = round(batter_df.apply(lambda x: 100*(x['runs']/x['balls']), axis = 1),2) # calculate strike rate

    batter_df['BPD'] = round(batter_df.apply(lambda x: balls_per_dismissal(x['balls'], x['player_dismissed']), axis = 1),2) # calculate balls per dismissal

    batter_df['BPB'] = round(batter_df.apply(lambda x: balls_per_boundary(x['balls'], x['fours']+x['sixes']), axis = 1),2) # calculate balls per boundary

    batter_df['dot_percentage'] = round(batter_df.apply(lambda x: 100*(x['dots']/x['balls']), axis = 1), 2) # calculate dot ball percentage

    return batter_df # return the dataframe

In [ ]:
dfp = ByCustom(df, 'Powerplay', 1) # get the dataframe for powerplay phase
dfmo = ByCustom(df, 'Middle Overs', 1) # get the dataframe for middle overs phase
dfdo = ByCustom(df, 'Death Overs', 1) # get the dataframe for death overs phase

In [ ]:
dfp2 = ByCustom(df, 'Powerplay', 2) # get the dataframe for powerplay phase
dfmo2 = ByCustom(df, 'Middle Overs', 2) # get the dataframe for middle overs phase
dfdo2 = ByCustom(df, 'Death Overs', 2) # get the dataframe for death overs phase

In [ ]:
def identify_elite_batters(df, min_innings=15, sr_thresh=80, rpi_thresh=80, bpb_thresh=80, dot_pct_thresh=20):
    """
    Identifies elite batters based on percentile ranks for Strike Rate (SR), 
    Runs Per Innings (RPI), Balls Per Boundary (BPB), and Dot Percentage.
    
    Parameters:
    - df: Pandas DataFrame containing batting statistics
    - min_innings: Minimum innings played (default = 15)
    - sr_thresh: Percentile threshold for SR (higher is better, default = 80)
    - rpi_thresh: Percentile threshold for RPI (higher is better, default = 80)
    - bpb_thresh: Percentile threshold for BPB (lower is better, default = 80)
    - dot_pct_thresh: Percentile threshold for dot percentage (lower is better, default = 20)

    Returns:
    - elite_batters: DataFrame with batters meeting at least 3 of the 4 conditions
    """

    # Calculate Percentile Ranks
    df['SR_percentile'] = df['SR'].rank(pct=True) * 100
    df['RPI_percentile'] = df['RPI'].rank(pct=True) * 100
    df['BPB_percentile'] = df['BPB'].rank(pct=True) * 100
    df['dot_pct_percentile'] = df['dot_percentage'].rank(pct=True) * 100

    # Count How Many Conditions Each Batter Meets
    df['num_conditions_met'] = (
        (df['SR_percentile'] > sr_thresh).astype(int) +  # Higher is better
        (df['RPI_percentile'] > rpi_thresh).astype(int) +  # Higher is better
        (df['BPB_percentile'] < bpb_thresh).astype(int) +  # Lower is better
        (df['dot_pct_percentile'] < dot_pct_thresh).astype(int)  # Lower is better
    )

    # Apply Filters
    elite_batters = df[(df['innings'] >= min_innings) & (df['num_conditions_met'] >= 3)]
    
    return elite_batters # Return DataFrame with elite batters


In [ ]:
def identify_underrated_batters(df, min_innings=10, max_innings=25, sr_thresh=70, rpi_thresh=70, bpb_thresh=70, dot_pct_thresh=20):
    """
    Identifies underrated batters who have high performance in key metrics but have played limited innings.
    
    Parameters:
    - df: Pandas DataFrame containing batting statistics
    - min_innings: Minimum innings played to be considered (default = 10)
    - max_innings: Maximum innings to ensure the player is not already highly rated (default = 25)
    - sr_thresh: Percentile threshold for SR (higher is better, default = 80)
    - rpi_thresh: Percentile threshold for RPI (higher is better, default = 80)
    - bpb_thresh: Percentile threshold for BPB (lower is better, default = 80)
    - dot_pct_thresh: Percentile threshold for dot percentage (lower is better, default = 20)

    Returns:
    - underrated_batters: DataFrame with players meeting at least 3 of the 4 conditions but played under max_innings
    """

    # Calculate Percentile Ranks
    df['SR_percentile'] = df['SR'].rank(pct=True) * 100
    df['RPI_percentile'] = df['RPI'].rank(pct=True) * 100
    df['BPB_percentile'] = df['BPB'].rank(pct=True) * 100
    df['dot_pct_percentile'] = df['dot_percentage'].rank(pct=True) * 100

    # Count How Many Conditions Each Batter Meets
    df['num_conditions_met'] = (
        (df['SR_percentile'] > sr_thresh).astype(int) +  # Higher is better
        (df['RPI_percentile'] > rpi_thresh).astype(int) +  # Higher is better
        (df['BPB_percentile'] < bpb_thresh).astype(int) +  # Lower is better
        (df['dot_pct_percentile'] < dot_pct_thresh).astype(int)  # Lower is better
    )

    # Apply Filters for Underrated Players
    underrated_batters = df[(df['innings'] >= min_innings) & (df['innings'] <= max_innings) & (df['num_conditions_met'] >= 3)]
    
    return underrated_batters


In [33]:
underrated_batters = identify_underrated_batters(dfp2).sort_values(by='RPI', ascending=False)
underrated_batters

,batter,runs,balls,innings,player_dismissed,sixes,fours,dots,RPI,SR,BPD,BPB,dot_percentage,SR_percentile,RPI_percentile,BPB_percentile,dot_pct_percentile,num_conditions_met
398,PD Salt,488,294,20,12,24,56,113,24.40,165.99,24.50,3.68,38.44,94.772344,97.133221,14.671164,10.118044,4
553,TM Head,313,192,13,8,12,43,84,24.08,163.02,24.00,3.49,43.75,94.435076,96.964587,13.322091,19.477234,4
161,G Gambhir,322,251,15,10,3,49,112,21.47,128.29,25.10,4.83,44.62,81.112985,95.952782,27.150084,21.500843,3
142,E Lewis,480,349,23,14,31,45,179,20.87,137.54,24.93,4.59,51.29,87.352445,95.278246,23.440135,46.543002,3
394,P Nissanka,401,339,20,8,11,49,162,20.05,118.29,42.38,5.65,47.79,72.175379,94.603710,39.460371,30.522766,3
322,MJ Lumb,256,197,13,8,11,29,98,19.69,129.95,24.62,4.92,49.75,83.136594,93.423272,28.330523,36.593592,3
362,N Dickwella,251,175,13,10,6,37,80,19.31,143.43,17.50,4.07,45.71,89.376054,92.917369,19.898820,24.957841,3
471,SD Hope,325,222,17,8,19,31,95,19.12,146.40,27.75,4.44,42.79,90.556492,92.580101,21.669477,16.526138,4
477,SIR Dunkley,411,318,22,12,9,59,154,18.68,129.25,26.50,4.68,48.43,82.630691,91.399663,25.548061,32.883642,3
125,DJM Short,244,206,14,6,6,33,101,17.43,118.45,34.33,5.28,49.03,72.344013,89.207420,35.244519,34.738617,3
